In [2]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
# import pyclifford
from pyclifford import utils, paulialg, stabilizer
from pyclifford.utils import acq, ipow, clifford_rotate, pauli_transform
from pyclifford.paulialg import Pauli, PauliList, PauliPolynomial

In [42]:
def init_stabilizer_state(n, t, otcm, mask=None):
    ''' Compute the 
    Parameters:
    n: (int) - number of physical qubits
    t: (int) - number of ancilla qubits
    otcm: boolean (w) - outcome bit string of length w (currently only support w==n)
    mask: boolean (N) - a boolean array specifying the subset of qubits outcome has support on
    
    Returns:
    init_st: initialized stabilizer state, rank=n+
    '''
    assert n >= 1 and t >= 1
    stbs = []
    if mask == None:
        assert len(otcm) == n
        for i, xi in enumerate(otcm):
            stb = 'I'*i + 'Z' + 'I'*(n-1-i+t)
            if xi == 1:
                stb = '-' + stb
            stbs.append(stb)
        for j in range(t):
            stb = 'I'*(n+j) + 'Z' + 'I'*(t-1-j)
            stbs.append(stb)
    else:
        assert len(otcm) == np.sum(mask)
        pass
    return stabilizer.stabilizer_state(stbs)

For example, mask=None, otcm=[1, 0, 1, 0].
Then $R_0=-Z_0, R_1=Z_1, R_2=-Z_2, R_3=Z_3, R_4=Z_4, R_5=Z_5$.

#### Test initial pure state

In [43]:
init_st = init_stabilizer_state(4, 2, np.array([1, 0, 1, 0]))
init_st

StabilizerState(
   -ZIIIII
   +IZIIII
   -IIZIII
   +IIIZII
   +IIIIZI
   +IIIIIZ)

We then use a manually-chosen fixed circuit that contains $t=2$ T gates to evolve these stabilizers,
$$R_j^{(V)}=V^\dagger R_j V$$

One problem is how to realize CNOT, Hadamard etc using PauliAlgebra methods. Naively, we use the decomposition
$H=\frac{1}{\sqrt{2}}(X+Z)$, $\mathrm{CNOT}=\frac{1}{2}(XX+IX+ZI-ZX)$.

Let's consider a simple circuit $V$:

+ 1st layer: Hadamard[0],[1],[2],[3]

+ 2nd layer: CNOT[0,4], CNOT[2,5]

+ 3rd layer: computational basis measurement[0],[1],[2],[3]

#### Define gadgetized circuit $V$

In [21]:
# represent Hadamard layer by Clifford map
hadamard_layer = stabilizer.stabilizer_state('XIIIII','IXIIII','IIXIII','IIIXII','IIIIZI','IIIIIZ').to_map()
hadamard_layer

CliffordMap(
  X0-> +ZIIIII
  Z0-> +XIIIII
  X1-> +IZIIII
  Z1-> +IXIIII
  X2-> +IIZIII
  Z2-> +IIXIII
  X3-> +IIIZII
  Z3-> +IIIXII
  X4-> +IIIIXI
  Z4-> +IIIIZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

In [22]:
# represent CNOT layer by Clifford map
cnot_layer = stabilizer.stabilizer_state('ZIIIII','IZIIII','IIZIII','IIIZII','ZIIIZI','IZIIIZ').to_map()
cnot_layer

CliffordMap(
  X0-> +XIIIXI
  Z0-> +ZIIIII
  X1-> +IXIIIX
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +IIIXII
  Z3-> +IIIZII
  X4-> +IIIIXI
  Z4-> +ZIIIZI
  X5-> +IIIIIX
  Z5-> +IZIIIZ)

In [33]:
# compose layers
V = hadamard_layer.compose(cnot_layer)

In [34]:
# evolve initial stabilizers
V_st = init_st.transform_by(V)

#### calculate $v$

In [48]:
v = 0
for i in range(V_st.r, len(V_st)//2):
    stb = V_st.gs[i]
    flag = True  # assume this stabilizer only contains I or Z
    for j in range(len(stb)//2):
        if stb[2*j] == 1:  # this stabilizer contains X or Y
            flag = False
    if flag == True:
        v += 1

In [49]:
v

6

#### obtain the projector for stabilizer group $G$

Now we already have the state $V^\dagger \Pi(x) V$, we want to know the state $\langle 0^n|V^\dagger \Pi(x) V |0^n\rangle$

This actually needs to be combined into the process of counting $v$: For one of the $n$ physical qubits, if the stabilizer on current qubit is $I$ or $Z$, keep this stabilizer, erase this qubit. At last, if there is an overall $1j^{\mathrm{ipow}}$ factor, add it.